# Carga de Datos - California Housing Dataset

Este notebook se encarga de la carga inicial de datos desde el archivo CSV.

**Objetivo**: Cargar el dataset y realizar una exploración inicial básica.

**Dataset**: California Housing - Predicción de precios de viviendas en California

**Autor**: MLOps Pipeline Project  
**Fecha**: Noviembre 2025

## 1. Importar Librerías Necesarias

In [1]:
import pandas as pd
import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')

# Para visualización básica
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

print("✓ Librerías importadas exitosamente")
print(f"Versión de pandas: {pd.__version__}")
print(f"Versión de numpy: {np.__version__}")

✓ Librerías importadas exitosamente
Versión de pandas: 2.3.3
Versión de numpy: 2.3.4


## 2. Cargar Configuración del Proyecto

In [2]:
# Cargar configuración desde config.json
with open('../config.json', 'r') as f:
    config = json.load(f)

print("=" * 80)
print("CONFIGURACIÓN DEL PROYECTO")
print("=" * 80)
for key, value in config.items():
    print(f"{key:20s}: {value}")
print("=" * 80)

CONFIGURACIÓN DEL PROYECTO
project_code        : mlops-pipeline
project_name        : MLOps Pipeline - California Housing
data_file           : Base_de_datos.csv
model_output_dir    : models/
random_state        : 42


## 3. Cargar el Dataset

In [3]:
# Cargar el dataset desde el archivo CSV
data_path = f"../{config['data_file']}"
df = pd.read_csv(data_path)

print("=" * 80)
print("DATASET CARGADO EXITOSAMENTE")
print("=" * 80)
print(f"Archivo: {config['data_file']}")
print(f"Dimensiones: {df.shape[0]:,} filas × {df.shape[1]} columnas")
print(f"Tamaño en memoria: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print("=" * 80)

DATASET CARGADO EXITOSAMENTE
Archivo: Base_de_datos.csv
Dimensiones: 20,640 filas × 10 columnas
Tamaño en memoria: 2.54 MB
